In [ ]:
import numpy
from keras import backend as K
from keras.datasets import mnist
from keras.utils import np_utils
from keras.layers import Dense, Dropout,Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
import pandas as pd

model = Sequential()
K.set_image_data_format('channels_last')
numpy.random.seed(0)
X = pd.read_csv('digit-recognizer/train.csv')
test = pd.read_csv('digit-recognizer/test.csv')

y = X["label"]
X.drop(["label"], inplace = True, axis = 1)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2 , random_state=42)

X_train = X_train.reshape(X_train.shape[0], 28, 28 , 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28 , 1).astype('float32')

In [ ]:
import matplotlib.pyplot as plt
print("the number of training examples = %i" % X_train.shape[0])
print("the number of classes = %i" % len(numpy.unique(y_train)))
print("Dimention of images = {:d} x {:d}  ".format(X_train[1].shape[0],X_train[1].shape[1])  )

#This line will allow us to know the number of occurrences of each specific class in the data
unique, count= numpy.unique(y_train, return_counts=True)
print("The number of occurance of each class in the dataset = %s " % dict (zip(unique, count) ), "\n" )

images_and_labels = list(zip(X_train,  y_train))
for index, (image, label) in enumerate(images_and_labels[:4]):
    plt.subplot(5, 4, index + 1)
    plt.axis('off')
    plt.imshow(image.squeeze(), cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('label: %i' % label )
plt.show()

In [ ]:
from keras.layers import Dropout
from keras.layers.core import Activation

model.add(Conv2D(20, kernel_size=5, padding="same",input_shape=(28, 28, 1), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(20, kernel_size=3, padding="same", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, kernel_size=3, padding="valid", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(units=10, activation='relu'  ))
model.add(Dropout(0.1))
model.add(Dense(10))
model.add(Activation("softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

y_train = np_utils.to_categorical(y_train).astype('int32')
y_test = np_utils.to_categorical(y_test)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

from keras.preprocessing.image import ImageDataGenerator
X_train2 = numpy.array(X_train, copy=True) 
y_train2 = numpy.array(y_train, copy=True) 

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
)

datagen.fit(X_train)

# Concatenating the old data with the augmented data
result_x  = numpy.concatenate((X_train, X_train2), axis=0)
result_y  = numpy.concatenate((y_train, y_train2), axis=0)


# # fits the model on batches with real-time data augmentation:
history = model.fit_generator(datagen.flow(result_x, result_y, batch_size=128),
                    steps_per_epoch=len(X_train) / 32, epochs = 20)

In [ ]:
test_set = (test.values).reshape(-1, 28, 28 , 1).astype('float32')

res = model.predict(test_set)
res = numpy.argmax(res,axis = 1)
res = pd.Series(res, name="Label")
submission = pd.concat([pd.Series(range(1 ,28001) ,name = "ImageId"),   res],axis = 1)
submission.to_csv("cnn_mnist_datagen.csv",index=False)
submission.head(10)